# AV Scheduling for household

### Initialize
* Inital functions and package
* Read input

In [87]:
import numpy as np
import pandas as pd
import matplotlib as plt
import networkx as nx
%matplotlib inline
# from DYNASMART_Process import *
from Modules import DYNASMART_Process as dy
# import preprocessing_read_skim_file as rs
from Modules import AV_functions as av
from Modules import Solve_DARP as sod
from Modules import Postprocess_DARP as pod
from Modules import Preprocess_DARP as prd
from Modules import DARP_to_DYNA as dtd
from gurobipy import *
from Modules import *
import datetime
import requests
import math
import importlib
import pickle
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#Read in the basic traveler and trips informationa and joint the houshold id
# traveler_info=pd.read_csv('input/traveler.csv')
personData=pd.read_csv('input/personData_1.csv')
person_type_dic=dict(zip(personData.person_id,personData.type))
# hh_temp=personData[['person_id','hh_id']]
# traveler_trips=pd.merge(traveler_info,hh_temp,how='left',on=['person_id'])
#Trip mode
#1 Drive alone free; 2 Drive alone pay; 3 Shared ride 2 free; 4 Shared ride 2 pay ;
#5 Shared ride 3+ free; 6 Shared ride 3+ pay; 7 Walk; 8 Bike; 9 Walk to local transit; 
#10 Walk to premium transit; 11 Drive to local transit; 12 Drive to premium transit; 
#13 Taxi; 14 School bus

# traveler_trips=pd.read_csv('input/traveler_trip_info.csv')
# #Calculate the expected trip travel time as trip start time of trip (i+1) - start time of trip i - activity time 
# traveler_trips['Diff_bet_starttime']=traveler_trips.starttime.diff()[1:]
# traveler_trips.Diff_bet_starttime=traveler_trips.Diff_bet_starttime.shift(-1)
# traveler_trips['Diff_bet_starttime'].apply(lambda x: 0 if x<0 else x)
# traveler_trips['travel_time']=traveler_trips.Diff_bet_starttime-traveler_trips.ActivityTime
# traveler_trips.loc[traveler_trips.trip_counter==traveler_trips.num_trips,'travel_time']=1440-(traveler_trips.loc[traveler_trips.trip_counter==traveler_trips.num_trips]['starttime']+traveler_trips.loc[traveler_trips.trip_counter==traveler_trips.num_trips]['ActivityTime'])
# traveler_trips.orig_purpose.replace(['eatingout','work','discretionary','shop','work-based','work-related','escort','maintenance','school','visiting','university'],
#                                     ['EatingOut','Work','Discretionary','Shop','WorkBased','WorkBased','Escort','Maintenance','School','Visiting','University'],
#                                     inplace=True)
# traveler_trips.dest_purpose.replace(['eatingout','work','discretionary','shop','work-based','work-related','escort','maintenance','school','visiting','university'],
#                                     ['EatingOut','Work','Discretionary','Shop','WorkBased','WorkBased','Escort','Maintenance','School','Visiting','University'],
#                                     inplace=True)
# traveler_trips.loc[:,'person_type']=traveler_trips.person_id.apply(lambda x: person_type_dic[x])
# value_latearr_penalty_per_min=pd.read_csv('input/LateArrPenalty.dat',sep='\t',index_col =0)
# value_earlyarr_penalty_per_min=pd.read_csv('input/EarlyArrPenalty.dat',sep='\t',index_col =0)
# early_penalty_threshold=pd.read_csv('input/EarlyArrPenaltyThreshold.dat',sep='\t',index_col=0)
# late_penalty_threshold=pd.read_csv('input/LateArrPenaltyThreshold.dat',sep='\t',index_col=0)

# traveler_trips.loc[:,'earlyarr_penalty']=traveler_trips.apply(lambda row: value_earlyarr_penalty_per_min.loc[row.dest_purpose,row.person_type], axis=1)
# traveler_trips.loc[:,'latearr_penalty']=traveler_trips.apply(lambda row: value_latearr_penalty_per_min.loc[row.dest_purpose,row.person_type], axis=1)
# traveler_trips.loc[:,'early_penalty_threshold']=traveler_trips.apply(lambda row: early_penalty_threshold.loc[row.dest_purpose,row.person_type], axis=1)
# traveler_trips.loc[:,'late_penalty_threshold']=traveler_trips.apply(lambda row: late_penalty_threshold.loc[row.dest_purpose,row.person_type], axis=1)
# traveler_trips.to_csv('input/traveler_trip_info_improved.csv')
traveler_trips=pd.read_csv('input/traveler_trip_info_improved.csv')
num_zones,num_nodes,num_links,node_detail,node_id,link_detail,link_id=dy.read_network('input/network.dat')
nodexy=dy.read_xy('input/xy.dat')
superzone_filepath='Input/SuperZone.dat'
linktraveltime_filepath='Input/output_td_linktraveltime.dat'
superzone_map=dy.read_superzone_info(superzone_filepath)
origin_path='Input/origin.dat'
destination_path='Input/destination.dat'
origins,destinations,origin_links=dy.read_origin_destion(origin_path,destination_path)
origin_links['length']=origin_links.apply(lambda row: 
                                          link_detail[int(link_id[node_id[row.u_node],node_id[row.d_node]])][0,4],axis=1)
VPNRSkim_folder_filepath='Input/vehicularPNRSkim'
Vehicular_Skim,Vehicular_Skim_Dict=dy.read_vehicle_skim(VPNRSkim_folder_filepath)
transit_skim_folderpath='Input/'
TransitMazTazFlag,drivingweight,walkingweight,TransitSkimTimeIntervalLength,MaxNumTransitSkimTimeInterval,WalkSpeed,transit_zone_candidates,Transit_AB_Cost_Skim,Transit_AB_Time_Skim,three_link_walk=dy.read_transitskim(transit_skim_folderpath)
Transit_AB_Cost_Skim_Dict=dy.convert_skim_dataframe_to_dict(Transit_AB_Cost_Skim,TransitSkimTimeIntervalLength)
Transit_AB_Time_Skim_Dict=dy.convert_skim_dataframe_to_dict(Transit_AB_Time_Skim,TransitSkimTimeIntervalLength)
transit_zone_dict={}
for link_zone in three_link_walk.three_link_zone.unique():
    transit_zone_dict[link_zone]=list(three_link_walk[three_link_walk.three_link_zone==link_zone]['transit_zone'].astype(int))

    three_link_walk_dict={}
for index,group in three_link_walk.groupby(['three_link_zone']):
    three_link_walk_dict[index]={}
    for inx,row in group.iterrows():
        three_link_walk_dict[index][row.transit_zone]=row.distance
#origins,destinations=dy.read_origin_destion('input/origin.dat','input/destination.dat')

56 57 72


In [ ]:
# Test the schedule adjustment solve with exact method
reward_mode=2
num_hh_member,hh_num_trips,C,TT,expected_arrival_time,expected_leave_time,early_penalty,late_penalty,early_penalty_threshold,late_penalty_threshold,visit_candidate_zone=prd.extract_hh_information(sorted_trips,Vehicular_Skim,Transit_AB_Cost_Skim,superzone_map,drivingcost_per_mile,num_time_interval)
R=prd.estimate_trip_reward(hh_num_trips,sorted_trips,Vehicular_Skim,reward_mode,superzone_map,drivingcost_per_mile)
num_cav=1
run_mode=2
single_model_runtime=60*30
m1,x,y,T,obj1,obj2,obj3=sod.dial_n_ride_model_schedule_adjustment(num_hh_member,hh_num_trips,C,TT,TL,TU,sorted_trips,
                    expected_arrival_time,early_penalty,late_penalty,
                    early_penalty_threshold,late_penalty_threshold,R,Vehicular_Skim,
                    share_ride_factor,output_flag,run_mode,reward_mode,
                    num_cav,num_time_interval,cav_use_mode,time_window_flag,single_model_runtime)

In [ ]:
output_flag=1
target_hh_id=2287301
# target_hh_id=2744264
min_length=15
max_length=20
single_model_runtime=60*0.5
drivingcost_per_mile=1
reward_mode=2
run_mode=0
darp_solutions=[]
cav_use_mode=1
num_time_interval=1
TL=[i*1440/num_time_interval for i in range(num_time_interval)]
TU=[(i+1)*1440/num_time_interval for i in range(num_time_interval)]
target_hh=traveler_trips[traveler_trips['hh_id']==target_hh_id].drop_duplicates(subset=['orig_maz','dest_maz','orig_purpose','dest_purpose','starttime','joint_trip_flag'])
#Sort all trips based on start time. This step could reduce the solving time and make it easier to track
sorted_trips=target_hh.sort_values("starttime")
#hh_index give an index to all trips within the household for tracking purpose
hh_num_trips=sorted_trips.shape[0]
sorted_trips["hh_index"]=(range(hh_num_trips))
# sorted_trips=sorted_trips.loc[sorted_trips.tripmode<=6]

num_cav=2
share_ride_factor=1.5
time_window_flag=0
#         time_window_flag=1
# num_cav=1
# share_ride_factor=1.5
# time_window_flag=0
print('**************')
print('Num_CAV',num_cav,share_ride_factor,time_window_flag)
print('**************')
darp_solutions.extend([sod.solve_with_schedule_partition(sorted_trips,Vehicular_Skim,Transit_AB_Cost_Skim,superzone_map,min_length,max_length,
                            reward_mode,drivingcost_per_mile,share_ride_factor,output_flag,run_mode,num_cav,
                          cav_use_mode,time_window_flag,single_model_runtime,num_time_interval,TL,TU)])

In [ ]:
sod.move_neighborhood(x_sol,T_sol,initial_route_info,hh_num_trips,
                      num_cav,shake_size,Vehicular_Skim_Dict)

In [ ]:
num_hh_member,hh_num_trips,C,TT,expected_arrival_time,expected_leave_time,early_penalty,late_penalty,early_penalty_threshold,late_penalty_threshold,visit_candidate_zone\
    =prd.extract_hh_information(sorted_trips,Vehicular_Skim,Transit_AB_Cost_Skim,superzone_map,drivingcost_per_mile,num_time_interval)

In [ ]:
print(datetime.datetime.now())
m3,T,S=sod.optimal_start_time(sorted_trips,x_sol[:,:,0],T_sol,hh_num_trips,expected_arrival_time,Vehicular_Skim_Dict,visit_candidate_zone,
    early_penalty,late_penalty,early_penalty_threshold,late_penalty_threshold,superzone_map)
print(datetime.datetime.now())

In [86]:
print(datetime.datetime.now())

for j in traveler_trips.orig_taz:
    z=1
print(datetime.datetime.now())


for index, row in traveler_trips.iterrows():
    z=1
    
print(datetime.datetime.now())

2018-07-12 18:45:05.555812
2018-07-12 18:45:05.614816
2018-07-12 18:45:37.005655


# Solve for all households car trips only

##### Compare the exact method and schedule method for one household

In [ ]:
output_flag=1
target_hh_id=364652
min_length=15
max_length=20
single_model_runtime=60*5
drivingcost_per_mile=1
reward_mode=2
run_mode=0
darp_solutions=[]
cav_use_mode=0
num_time_interval=1
TL=[i*1440/num_time_interval for i in range(num_time_interval)]
TU=[(i+1)*1440/num_time_interval for i in range(num_time_interval)]
num_cav=1
share_ride_factor=1.5
time_window_flag=0
print(datetime.datetime.now())
route_info1=sod.find_av_schedule_exact_method(target_hh_id,traveler_trips.loc[traveler_trips.tripmode<=6],output_flag,min_length,max_length,single_model_runtime,drivingcost_per_mile,
                         reward_mode,run_mode,cav_use_mode,num_time_interval,num_cav,share_ride_factor
                         ,time_window_flag,Vehicular_Skim_Dict,Transit_AB_Cost_Skim,superzone_map,TL,TU)
print(datetime.datetime.now())
target_hh=traveler_trips[(traveler_trips['hh_id']==target_hh_id) &(traveler_trips.tripmode<=6)]\
        .drop_duplicates(subset=['orig_maz','dest_maz','orig_purpose','dest_purpose',
                                 'starttime','joint_trip_flag'])
#Sort all trips based on start time. This step could reduce the solving time and make it easier to track
sorted_trips=target_hh.sort_values("starttime")
hh_num_trips=sorted_trips.shape[0]

sorted_trips["hh_index"]=(range(hh_num_trips))
print(datetime.datetime.now())
darp_solution1=sod.solve_with_schedule_partition(sorted_trips,Vehicular_Skim_Dict,Transit_AB_Cost_Skim,superzone_map,min_length,max_length,
                                    reward_mode,drivingcost_per_mile,share_ride_factor,output_flag,run_mode,num_cav,
                                    cav_use_mode,time_window_flag,single_model_runtime,num_time_interval,TL,TU)
print(datetime.datetime.now())

In [94]:
output_flag=0
# target_hh_id=2744264
min_length=15
max_length=20
single_model_runtime=60*3
drivingcost_per_mile=1
reward_mode=2
run_mode=0
darp_solutions=[]
cav_use_mode=0
num_time_interval=1
TL=[i*1440/num_time_interval for i in range(num_time_interval)]
TU=[(i+1)*1440/num_time_interval for i in range(num_time_interval)]
num_cav=1
share_ride_factor=1.5
time_window_flag=0
target_trips=traveler_trips.loc[(traveler_trips.tripmode<=6) &(traveler_trips.driver_passenger_flag>0)]
target_trips=target_trips[(target_trips.orig_taz!=target_trips.dest_taz) | (target_trips.trip_counter==1)]

route_info_allcartrips15,darp_solutions_allcartrips_temp15=\
sod.get_route_info_allhh(target_trips,
                         output_flag,min_length,max_length,single_model_runtime,drivingcost_per_mile,
                         reward_mode,run_mode,cav_use_mode,num_time_interval,num_cav,share_ride_factor
                         ,time_window_flag,Vehicular_Skim_Dict,Transit_AB_Cost_Skim,superzone_map,TL,TU,transit_zone_dict)

Estimate Route for the  0 th household  2018-07-12 22:13:44.295235
1 9 2018-07-12 22:13:44.596259
2 14 2018-07-12 22:13:45.471304
3 13 2018-07-12 22:13:45.988336
4 25 2018-07-12 22:13:47.606430
5 9 2018-07-12 22:13:47.966453
6 6 2018-07-12 22:13:48.247467
7 8 2018-07-12 22:13:48.629495
8 9 2018-07-12 22:13:49.062516
9 8 2018-07-12 22:13:49.422542
10 7 2018-07-12 22:13:49.785563
11 11 2018-07-12 22:13:50.822620
12 12 2018-07-12 22:13:51.279647
13 4 2018-07-12 22:13:51.519665
14 6 2018-07-12 22:13:51.836677
15 13 2018-07-12 22:13:52.485722
16 14 2018-07-12 22:13:53.038750
17 11 2018-07-12 22:13:57.687028
18 8 2018-07-12 22:13:57.989046
19 6 2018-07-12 22:13:58.255056
20 5 2018-07-12 22:13:58.646084
21 6 2018-07-12 22:13:58.926100
22 10 2018-07-12 22:13:59.383122
23 15 2018-07-12 22:14:00.028164
24 18 2018-07-12 22:14:00.895213
25 14 2018-07-12 22:14:02.949336
26 6 2018-07-12 22:14:03.220353
27 9 2018-07-12 22:14:03.628373
28 6 2018-07-12 22:14:03.925390
29 5 2018-07-12 22:14:04.194405
30

243 13 2018-07-12 22:16:33.458820
244 21 2018-07-12 22:16:34.925911
245 14 2018-07-12 22:16:35.675965
246 10 2018-07-12 22:16:36.085991
247 8 2018-07-12 22:16:36.413007
248 9 2018-07-12 22:16:36.783034
249 10 2018-07-12 22:16:37.231067
250 12 2018-07-12 22:16:37.982116
251 20 2018-07-12 22:16:39.318200
252 27 2018-07-12 22:16:42.192391
253 2 2018-07-12 22:16:42.338402
254 20 2018-07-12 22:16:45.470601
255 4 2018-07-12 22:16:45.684618
256 16 2018-07-12 22:16:46.780687
257 11 2018-07-12 22:16:47.406726
258 13 2018-07-12 22:16:48.687812
259 8 2018-07-12 22:16:49.024833
260 10 2018-07-12 22:16:49.421860
261 26 2018-07-12 22:16:52.456057
262 25 2018-07-12 22:17:11.537302
263 11 2018-07-12 22:17:12.029340
264 2 2018-07-12 22:17:12.168343
265 11 2018-07-12 22:17:12.685383
266 13 2018-07-12 22:17:13.194416
267 9 2018-07-12 22:17:13.544439
268 9 2018-07-12 22:17:14.000464
269 4 2018-07-12 22:17:14.192481
270 10 2018-07-12 22:17:14.574500
271 7 2018-07-12 22:17:14.864520
272 8 2018-07-12 22:17:1

484 8 2018-07-12 22:20:02.923139
485 4 2018-07-12 22:20:03.169153
486 14 2018-07-12 22:20:03.729180
487 10 2018-07-12 22:20:04.064200
488 12 2018-07-12 22:20:04.772242
489 7 2018-07-12 22:20:05.061259
490 10 2018-07-12 22:20:05.436288
491 9 2018-07-12 22:20:05.829309
492 14 2018-07-12 22:20:06.860367
493 8 2018-07-12 22:20:07.107384
494 7 2018-07-12 22:20:07.412397
495 7 2018-07-12 22:20:07.689414
496 7 2018-07-12 22:20:07.974430
497 5 2018-07-12 22:20:08.196444
498 6 2018-07-12 22:20:08.494461
499 9 2018-07-12 22:20:08.867485
500 11 2018-07-12 22:20:09.393521
Estimate Route for the  500 th household  2018-07-12 22:20:09.399519
501 7 2018-07-12 22:20:09.676536
502 14 2018-07-12 22:20:10.570588
503 18 2018-07-12 22:20:11.645651
504 20 2018-07-12 22:20:12.917723
505 10 2018-07-12 22:20:13.468759
506 11 2018-07-12 22:20:14.301808
507 5 2018-07-12 22:20:14.564818
508 11 2018-07-12 22:20:15.084854
509 8 2018-07-12 22:20:15.418873
510 2 2018-07-12 22:20:15.565883
511 13 2018-07-12 22:20:16.2

723 12 2018-07-12 22:22:19.037139
724 6 2018-07-12 22:22:19.221150
725 2 2018-07-12 22:22:19.365158
726 3 2018-07-12 22:22:19.510167
727 5 2018-07-12 22:22:19.785180
728 13 2018-07-12 22:22:21.633287
729 13 2018-07-12 22:22:22.313332
730 4 2018-07-12 22:22:22.484343
731 18 2018-07-12 22:22:24.615461
732 6 2018-07-12 22:22:24.906484
733 14 2018-07-12 22:22:26.757587
734 13 2018-07-12 22:22:28.044669
735 10 2018-07-12 22:22:28.601702
736 13 2018-07-12 22:22:29.273741
737 19 2018-07-12 22:22:30.039786
738 10 2018-07-12 22:22:30.480810
739 2 2018-07-12 22:22:30.622820
740 17 2018-07-12 22:22:31.668881
741 6 2018-07-12 22:22:31.924891
742 9 2018-07-12 22:22:32.323915
743 9 2018-07-12 22:22:32.728938
744 10 2018-07-12 22:22:33.211973
745 14 2018-07-12 22:22:33.762001
746 7 2018-07-12 22:22:33.989018
747 12 2018-07-12 22:22:34.517049
748 6 2018-07-12 22:22:34.764064
749 13 2018-07-12 22:22:35.509103
750 3 2018-07-12 22:22:35.684113
751 10 2018-07-12 22:22:36.133138
752 9 2018-07-12 22:22:36.4

965 17 2018-07-12 22:24:44.224673
966 10 2018-07-12 22:24:44.704701
967 15 2018-07-12 22:24:46.645809
968 7 2018-07-12 22:24:46.958833
969 8 2018-07-12 22:24:47.356857
970 3 2018-07-12 22:24:47.513867
971 15 2018-07-12 22:24:48.388918
972 8 2018-07-12 22:24:48.650933
973 5 2018-07-12 22:24:48.866940
974 11 2018-07-12 22:24:49.854004
975 6 2018-07-12 22:24:50.122019
976 4 2018-07-12 22:24:50.293029
977 20 2018-07-12 22:24:53.591220
978 10 2018-07-12 22:24:54.017248
979 5 2018-07-12 22:24:54.289264
980 9 2018-07-12 22:24:54.700288
981 19 2018-07-12 22:24:56.407383
982 19 2018-07-12 22:24:57.183434
983 20 2018-07-12 22:25:01.060656
984 5 2018-07-12 22:25:01.298676
985 7 2018-07-12 22:25:01.598694
986 2 2018-07-12 22:25:01.745696
987 10 2018-07-12 22:25:02.934767
988 6 2018-07-12 22:25:03.243791
989 16 2018-07-12 22:25:04.128843
990 8 2018-07-12 22:25:04.458862
991 5 2018-07-12 22:25:04.705876
992 11 2018-07-12 22:25:05.885947
993 7 2018-07-12 22:25:06.167963
994 15 2018-07-12 22:25:07.322

...

28689 2 2018-07-13 01:46:30.909176
28690 2 2018-07-13 01:46:31.165191
28691 2 2018-07-13 01:46:31.416206
28692 2 2018-07-13 01:46:31.669221
28693 2 2018-07-13 01:46:31.922236
28694 4 2018-07-13 01:46:32.225254
28695 2 2018-07-13 01:46:32.475269
28696 2 2018-07-13 01:46:32.730283
28697 5 2018-07-13 01:46:33.073303
28698 2 2018-07-13 01:46:33.327318
28699 4 2018-07-13 01:46:33.842349
28700 3 2018-07-13 01:46:34.128366
Estimate Route for the  28700 th household  2018-07-13 01:46:34.244372
28701 4 2018-07-13 01:46:34.475386
28702 2 2018-07-13 01:46:34.727401
28703 3 2018-07-13 01:46:34.990416
28704 2 2018-07-13 01:46:35.244431
28705 5 2018-07-13 01:46:35.584451
28706 10 2018-07-13 01:46:36.283493
28707 2 2018-07-13 01:46:36.536507
28708 4 2018-07-13 01:46:36.872527
28709 2 2018-07-13 01:46:37.124542
28710 2 2018-07-13 01:46:37.378556
28711 2 2018-07-13 01:46:37.630571
28712 1 2018-07-13 01:46:37.836583
28713 2 2018-07-13 01:46:38.088598
28714 5 2018-07-13 01:46:38.426619
28715 2 2018-07-13

28917 4 2018-07-13 01:47:32.529798
28918 1 2018-07-13 01:47:32.736810
28919 2 2018-07-13 01:47:32.986825
28920 1 2018-07-13 01:47:33.196837
28921 2 2018-07-13 01:47:33.448852
28922 3 2018-07-13 01:47:33.703867
28923 7 2018-07-13 01:47:34.109891
28924 2 2018-07-13 01:47:34.366906
28925 2 2018-07-13 01:47:34.617921
28926 1 2018-07-13 01:47:34.823933
28927 1 2018-07-13 01:47:35.030945
28928 5 2018-07-13 01:47:35.377965
28929 2 2018-07-13 01:47:35.584978
28930 6 2018-07-13 01:47:35.898996
28931 4 2018-07-13 01:47:36.205013
28932 2 2018-07-13 01:47:36.457028
28933 2 2018-07-13 01:47:36.709043
28934 3 2018-07-13 01:47:36.971059
28935 2 2018-07-13 01:47:37.223074
28936 4 2018-07-13 01:47:37.504090
28937 2 2018-07-13 01:47:37.758105
28938 3 2018-07-13 01:47:38.019121
28939 3 2018-07-13 01:47:38.255135
28940 1 2018-07-13 01:47:38.462147
28941 2 2018-07-13 01:47:38.713156
28942 5 2018-07-13 01:47:39.054182
28943 2 2018-07-13 01:47:39.307196
28944 2 2018-07-13 01:47:39.528210
28945 2 2018-07-13 0

29148 3 2018-07-13 01:48:34.370432
29149 1 2018-07-13 01:48:34.577445
29150 2 2018-07-13 01:48:34.808458
29151 3 2018-07-13 01:48:35.069474
29152 1 2018-07-13 01:48:35.276486
29153 2 2018-07-13 01:48:35.530501
29154 2 2018-07-13 01:48:35.784516
29155 2 2018-07-13 01:48:36.039531
29156 2 2018-07-13 01:48:36.290546
29157 1 2018-07-13 01:48:36.495558
29158 4 2018-07-13 01:48:36.839578
29159 2 2018-07-13 01:48:37.090592
29160 2 2018-07-13 01:48:37.343607
29161 2 2018-07-13 01:48:37.596622
29162 4 2018-07-13 01:48:37.851638
29163 2 2018-07-13 01:48:38.110652
29164 3 2018-07-13 01:48:38.373668
29165 2 2018-07-13 01:48:38.577680
29166 3 2018-07-13 01:48:38.841696
29167 2 2018-07-13 01:48:39.093711
29168 5 2018-07-13 01:48:39.743749
29169 3 2018-07-13 01:48:39.999763
29170 2 2018-07-13 01:48:40.251779
29171 4 2018-07-13 01:48:40.555796
29172 2 2018-07-13 01:48:40.813812
29173 2 2018-07-13 01:48:41.064826
29174 2 2018-07-13 01:48:41.320841
29175 1 2018-07-13 01:48:41.529854
29176 2 2018-07-13 0

29379 5 2018-07-13 01:49:35.328016
29380 4 2018-07-13 01:49:35.629033
29381 2 2018-07-13 01:49:35.842046
29382 2 2018-07-13 01:49:36.094056
29383 6 2018-07-13 01:49:36.416080
29384 1 2018-07-13 01:49:36.624092
29385 2 2018-07-13 01:49:36.875107
29386 2 2018-07-13 01:49:37.128122
29387 2 2018-07-13 01:49:37.377136
29388 5 2018-07-13 01:49:37.654152
29389 2 2018-07-13 01:49:37.860165
29390 3 2018-07-13 01:49:38.146181
29391 3 2018-07-13 01:49:38.440198
29392 2 2018-07-13 01:49:38.691214
29393 2 2018-07-13 01:49:38.999231
29394 2 2018-07-13 01:49:39.293248
29395 3 2018-07-13 01:49:39.540260
29396 3 2018-07-13 01:49:39.840281
29397 4 2018-07-13 01:49:40.183301
29398 2 2018-07-13 01:49:40.435315
29399 2 2018-07-13 01:49:40.687331
29400 2 2018-07-13 01:49:40.940346
Estimate Route for the  29400 th household  2018-07-13 01:49:41.056348
29401 2 2018-07-13 01:49:41.198361
29402 2 2018-07-13 01:49:41.450376
29403 4 2018-07-13 01:49:41.756393
29404 1 2018-07-13 01:49:41.963405
29405 2 2018-07-13 

29607 2 2018-07-13 01:50:33.112412
29608 2 2018-07-13 01:50:33.364427
29609 3 2018-07-13 01:50:33.643443
29610 4 2018-07-13 01:50:33.947461
29611 2 2018-07-13 01:50:34.174474
29612 2 2018-07-13 01:50:34.397487
29613 2 2018-07-13 01:50:34.651502
29614 2 2018-07-13 01:50:34.906517
29615 2 2018-07-13 01:50:35.155532


In [98]:
import pickle
run_name='allhhcartrip1.5/'
output_filepath='Output/'+run_name
if not os.path.exists(output_filepath):
    os.makedirs(output_filepath)
route_info_allcartrips15.to_csv(output_filepath+'route_info.csv')
pod.save_obj(darp_solutions_allcartrips_temp15,'darp_solutions',output_filepath)

In [ ]:
darp_solutions_allcartrips_temp15

In [ ]:

if not os.path.exists(route_info_path):
    os.makedirs(route_info_path)
route_info_allcartrips15.to_csv(route_info_path+'route_info.csv')

vehicle_filepath=output_filepath+'vehicle.dat'
path_filepath=output_filepath+'path.dat'
intrasuperzone_vehicle_filepath='Input/intrasuperzone_vehicle.dat'
intrasuperzone_path_filepath='Input/intrasuperzone_path.dat'
intrasuperzone_info,intrasuperzone_path_dic=dtd.read_intrasuperzone_files(intrasuperzone_vehicle_filepath,intrasuperzone_path_filepath)
external_vehicle_filepath='Input/external_vehicle.dat'
average_value_of_time=round(traveler_trips.value_of_time.mean(),4)
dtd.route_to_vehiclepathdat(route_info_allcartrips,origin_links,vehicle_filepath,path_filepath,superzone_map,
                            intrasuperzone_path_dic,external_vehicle_filepath,average_value_of_time)

In [ ]:
output_flag=0
# target_hh_id=2744264
min_length=15
max_length=20
single_model_runtime=60*3
drivingcost_per_mile=1
reward_mode=1
run_mode=0
darp_solutions=[]
cav_use_mode=0
num_time_interval=1
TL=[i*1440/num_time_interval for i in range(num_time_interval)]
TU=[(i+1)*1440/num_time_interval for i in range(num_time_interval)]
num_cav=1
share_ride_factor=1.5
time_window_flag=0
target_trips=traveler_trips.loc[(traveler_trips.tripmode<=6) &(traveler_trips.driver_passenger_flag>0)]
target_trips=target_trips[(target_trips.orig_taz!=target_trips.dest_taz) | (target_trips.trip_counter==1)]
target_trips=target_trips.append(traveler_trips[traveler_trips.tripmode>6])
value_of_time_discount_factor=0.9
target_trips['value_of_time']=target_trips['value_of_time']*value_of_time_discount_factor

route_info_modechioce09,darp_solutions_modechoice09=\
sod.get_route_info_allhh(target_trips,
                         output_flag,min_length,max_length,single_model_runtime,drivingcost_per_mile,
                         reward_mode,run_mode,cav_use_mode,num_time_interval,num_cav,share_ride_factor
                         ,time_window_flag,Vehicular_Skim_Dict,
                         Transit_AB_Cost_Skim_Dict,Transit_AB_Time_Skim_Dict,three_link_walk_dict,
                         superzone_map,TL,TU,
                         transit_zone_dict,transit_zone_candidates,TransitMazTazFlag,
                         TransitSkimTimeIntervalLength )

Estimate Route for the  0 th household  2018-07-13 17:43:19.440563
1 12 2018-07-13 17:43:20.145611
2 26 2018-07-13 17:43:24.860885
3 19 2018-07-13 17:43:26.169959
4 31 2018-07-13 17:43:27.903067
5 11 2018-07-13 17:43:28.438099
6 20 2018-07-13 17:43:29.499156
7 11 2018-07-13 17:43:30.036189
8 11 2018-07-13 17:43:31.277263
9 8 2018-07-13 17:43:31.816297
10 19 2018-07-13 17:43:33.008363
11 13 2018-07-13 17:43:33.959417
12 16 2018-07-13 17:43:34.472453
13 12 2018-07-13 17:43:34.982484
14 11 2018-07-13 17:43:35.545513
15 24 2018-07-13 17:43:36.449564
16 16 2018-07-13 17:43:37.073605
17 17 2018-07-13 17:43:37.746645
18 12 2018-07-13 17:43:38.213668
19 10 2018-07-13 17:43:38.676700
20 7 2018-07-13 17:43:38.959717
21 18 2018-07-13 17:43:39.683753
22 15 2018-07-13 17:43:40.748818
23 22 2018-07-13 17:43:41.693878
24 18 2018-07-13 17:43:42.364917
25 18 2018-07-13 17:43:43.629991
26 6 2018-07-13 17:43:43.834999
27 12 2018-07-13 17:43:44.321032
28 6 2018-07-13 17:43:44.572046
29 7 2018-07-13 17:43:

241 11 2018-07-13 17:47:52.858635
242 28 2018-07-13 17:47:57.971934
243 30 2018-07-13 17:48:04.947344
244 21 2018-07-13 17:48:06.112419
245 18 2018-07-13 17:48:07.053467
246 10 2018-07-13 17:48:07.551503
247 15 2018-07-13 17:48:08.452550
248 11 2018-07-13 17:48:09.069593
249 10 2018-07-13 17:48:09.543620
250 12 2018-07-13 17:48:10.380664
251 26 2018-07-13 17:48:11.426726
252 29 2018-07-13 17:48:14.961939
253 2 2018-07-13 17:48:15.104946
254 20 2018-07-13 17:48:19.246185
255 6 2018-07-13 17:48:19.497205
256 22 2018-07-13 17:48:20.758279
257 11 2018-07-13 17:48:21.361310
258 25 2018-07-13 17:48:22.368369
259 8 2018-07-13 17:48:22.789392
260 13 2018-07-13 17:48:23.466435
261 26 2018-07-13 17:48:28.009700
262 25 2018-07-13 17:48:48.375897
263 17 2018-07-13 17:48:49.532967
264 4 2018-07-13 17:48:49.785980
265 14 2018-07-13 17:48:50.709035
266 19 2018-07-13 17:48:51.676092
267 15 2018-07-13 17:48:52.695151
268 18 2018-07-13 17:48:53.728214
269 7 2018-07-13 17:48:54.000233
270 16 2018-07-13 1

480 17 2018-07-13 17:54:14.602077
481 20 2018-07-13 17:54:15.560129
482 21 2018-07-13 17:54:18.715314
483 14 2018-07-13 17:54:19.495365
484 12 2018-07-13 17:54:20.092400
485 4 2018-07-13 17:54:20.328413
486 22 2018-07-13 17:54:21.195464
487 12 2018-07-13 17:54:21.615485
488 17 2018-07-13 17:54:22.398536
489 9 2018-07-13 17:54:22.794558
490 22 2018-07-13 17:54:23.591601
491 13 2018-07-13 17:54:24.564662
492 18 2018-07-13 17:54:25.124691
493 17 2018-07-13 17:54:26.133755
494 9 2018-07-13 17:54:26.521777
495 8 2018-07-13 17:54:26.786793
496 7 2018-07-13 17:54:27.084805
497 8 2018-07-13 17:54:27.416829
498 15 2018-07-13 17:54:28.373887
499 13 2018-07-13 17:54:28.853915
500 15 2018-07-13 17:54:29.746968
Estimate Route for the  500 th household  2018-07-13 17:54:29.754967
501 12 2018-07-13 17:54:30.363003
502 18 2018-07-13 17:54:30.903033
503 18 2018-07-13 17:54:31.635077
504 20 2018-07-13 17:54:32.800146
505 14 2018-07-13 17:54:33.795203
506 21 2018-07-13 17:54:37.464415
507 5 2018-07-13 17

718 7 2018-07-13 17:57:52.626891
719 21 2018-07-13 17:57:53.928965
720 17 2018-07-13 17:57:54.639004
721 23 2018-07-13 17:57:55.358047
722 19 2018-07-13 17:57:56.167094
723 23 2018-07-13 17:57:59.756304
724 11 2018-07-13 17:58:00.613358
725 4 2018-07-13 17:58:00.872372
726 3 2018-07-13 17:58:01.031380
727 7 2018-07-13 17:58:01.342398
728 16 2018-07-13 17:58:02.582472
729 16 2018-07-13 17:58:03.490524
730 4 2018-07-13 17:58:03.665541
731 24 2018-07-13 17:58:05.245632
732 10 2018-07-13 17:58:05.672658
733 18 2018-07-13 17:58:07.421755
734 13 2018-07-13 17:58:09.049851
735 11 2018-07-13 17:58:10.347928
736 16 2018-07-13 17:58:10.920960
737 19 2018-07-13 17:58:11.680011
738 22 2018-07-13 17:58:12.704071
739 7 2018-07-13 17:58:13.067089
740 29 2018-07-13 17:58:14.507172
741 6 2018-07-13 17:58:14.769192
742 11 2018-07-13 17:58:15.316225
743 21 2018-07-13 17:58:16.350285
744 16 2018-07-13 17:58:16.891316
745 17 2018-07-13 17:58:17.786369
746 9 2018-07-13 17:58:18.082384
747 16 2018-07-13 17:5

959 22 2018-07-13 18:03:19.521099
960 19 2018-07-13 18:03:23.470331
961 8 2018-07-13 18:03:23.810353
962 7 2018-07-13 18:03:24.144372
963 14 2018-07-13 18:03:24.834411
964 13 2018-07-13 18:03:25.391445
965 20 2018-07-13 18:03:26.705522
966 12 2018-07-13 18:03:27.424564
967 15 2018-07-13 18:03:28.038600
968 12 2018-07-13 18:03:30.427741
969 10 2018-07-13 18:03:31.067784
970 3 2018-07-13 18:03:31.229793
971 17 2018-07-13 18:03:31.770825
972 16 2018-07-13 18:03:32.279856
973 9 2018-07-13 18:03:32.571867
974 17 2018-07-13 18:03:33.241906
975 8 2018-07-13 18:03:33.575930
976 8 2018-07-13 18:03:33.998950
977 27 2018-07-13 18:03:36.780113
978 12 2018-07-13 18:03:37.408150
979 12 2018-07-13 18:03:38.249200
980 14 2018-07-13 18:03:39.188257
981 21 2018-07-13 18:03:39.966306
982 19 2018-07-13 18:03:40.641346
983 20 2018-07-13 18:03:44.221553
984 16 2018-07-13 18:03:46.252671
985 7 2018-07-13 18:03:46.758700
986 2 2018-07-13 18:03:47.038716
987 14 2018-07-13 18:03:50.199902
988 10 2018-07-13 18:0

...

7252 11 2018-07-13 19:53:46.237553
7253 7 2018-07-13 19:53:46.666578
7254 7 2018-07-13 19:53:47.101604
7255 12 2018-07-13 19:53:48.725699
7256 9 2018-07-13 19:53:49.619755
7257 11 2018-07-13 19:53:50.228789
7258 4 2018-07-13 19:53:50.498803
7259 10 2018-07-13 19:53:51.868887
7260 9 2018-07-13 19:53:52.220903
7261 7 2018-07-13 19:53:52.617932
7262 6 2018-07-13 19:53:52.994953
7263 22 2018-07-13 19:54:13.900179
7264 20 2018-07-13 19:54:18.407446
7265 7 2018-07-13 19:54:18.814468
7266 20 2018-07-13 19:54:24.104780
7267 8 2018-07-13 19:54:24.445800
7268 4 2018-07-13 19:54:24.704818
7269 12 2018-07-13 19:54:25.497859
7270 2 2018-07-13 19:54:25.736877
7271 13 2018-07-13 19:54:26.783940
7272 6 2018-07-13 19:54:27.070961
7273 17 2018-07-13 19:54:27.715992
7274 8 2018-07-13 19:54:28.053010
7275 14 2018-07-13 19:54:29.435090
7276 17 2018-07-13 19:54:30.249140
7277 26 2018-07-13 19:54:33.699347
7278 11 2018-07-13 19:54:34.146373
7279 4 2018-07-13 19:54:34.473392
7280 6 2018-07-13 19:54:34.791411


7486 8 2018-07-13 19:56:51.046414
7487 9 2018-07-13 19:56:51.534444
7488 7 2018-07-13 19:56:51.922466
7489 12 2018-07-13 19:56:52.449497
7490 10 2018-07-13 19:56:52.876524
7491 21 2018-07-13 19:56:54.156598
7492 5 2018-07-13 19:56:54.434618
7493 2 2018-07-13 19:56:54.656631
7494 7 2018-07-13 19:56:54.970650
7495 5 2018-07-13 19:56:55.283668
7496 11 2018-07-13 19:56:55.863701
7497 2 2018-07-13 19:56:56.086711
7498 8 2018-07-13 19:56:56.486738
7499 16 2018-07-13 19:56:57.122777
7500 22 2018-07-13 19:57:02.646095
Estimate Route for the  7500 th household  2018-07-13 19:57:02.725100
7501 11 2018-07-13 19:57:03.352138
7502 9 2018-07-13 19:57:03.811170
7503 3 2018-07-13 19:57:04.041177
7504 10 2018-07-13 19:57:04.453207
7505 8 2018-07-13 19:57:04.848225
7506 7 2018-07-13 19:57:05.245260
7507 4 2018-07-13 19:57:05.568268
7508 9 2018-07-13 19:57:06.659332
7509 13 2018-07-13 19:57:07.217367
7510 3 2018-07-13 19:57:07.462384
7511 9 2018-07-13 19:57:07.936407
7512 9 2018-07-13 19:57:08.466443
751

7718 10 2018-07-13 19:59:33.820987
7719 6 2018-07-13 19:59:34.187003
7720 18 2018-07-13 19:59:35.169065
7721 5 2018-07-13 19:59:35.508085
7722 11 2018-07-13 19:59:36.123119
7723 2 2018-07-13 19:59:36.375138
7724 6 2018-07-13 19:59:36.728155
7725 13 2018-07-13 19:59:37.397196
7726 7 2018-07-13 19:59:37.820216
7727 7 2018-07-13 19:59:38.140240
7728 12 2018-07-13 19:59:39.628323
7729 23 2018-07-13 19:59:40.909397
7730 11 2018-07-13 19:59:41.318428
7731 22 2018-07-13 19:59:42.387490
7732 5 2018-07-13 19:59:42.742509
7733 9 2018-07-13 19:59:43.075531
7734 13 2018-07-13 19:59:43.812569
7735 11 2018-07-13 19:59:44.523616
7736 5 2018-07-13 19:59:44.863635
7737 4 2018-07-13 19:59:45.117650
7738 6 2018-07-13 19:59:45.411668
7739 14 2018-07-13 19:59:45.956695
7740 6 2018-07-13 19:59:46.352719
7741 10 2018-07-13 19:59:46.777743
7742 9 2018-07-13 19:59:47.248770
7743 21 2018-07-13 19:59:48.547847
7744 7 2018-07-13 19:59:48.925869
7745 16 2018-07-13 19:59:49.567913
7746 14 2018-07-13 19:59:51.899045

7952 9 2018-07-13 20:05:36.851323
7953 8 2018-07-13 20:05:37.375350
7954 7 2018-07-13 20:05:37.774373
7955 11 2018-07-13 20:05:38.842441
7956 6 2018-07-13 20:05:39.190455
7957 5 2018-07-13 20:05:39.494475
7958 7 2018-07-13 20:05:39.894502
7959 31 2018-07-13 20:05:50.578130
7960 11 2018-07-13 20:05:51.000151
7961 12 2018-07-13 20:05:51.904203
7962 6 2018-07-13 20:05:52.254225
7963 6 2018-07-13 20:05:52.520241
7964 8 2018-07-13 20:05:52.896265
7965 4 2018-07-13 20:05:53.175281
7966 8 2018-07-13 20:05:53.638310
7967 9 2018-07-13 20:05:54.126334
7968 9 2018-07-13 20:05:54.475356
7969 19 2018-07-13 20:05:55.307408
7970 6 2018-07-13 20:05:55.663424
7971 10 2018-07-13 20:05:56.075454
7972 15 2018-07-13 20:05:56.839499
7973 5 2018-07-13 20:05:57.146511
7974 18 2018-07-13 20:05:58.664600
7975 2 2018-07-13 20:05:58.885619
7976 3 2018-07-13 20:05:59.042622
7977 13 2018-07-13 20:05:59.510651
7978 4 2018-07-13 20:05:59.761670
7979 11 2018-07-13 20:06:00.443706
7980 8 2018-07-13 20:06:00.893733
7981

8186 3 2018-07-13 20:08:32.398636
8187 8 2018-07-13 20:08:32.928669
8188 5 2018-07-13 20:08:33.236686
8189 10 2018-07-13 20:08:33.763720
8190 14 2018-07-13 20:08:34.882786
8191 4 2018-07-13 20:08:35.297807
8192 11 2018-07-13 20:08:35.917845
8193 8 2018-07-13 20:08:36.677888
8194 16 2018-07-13 20:08:37.731951
8195 19 2018-07-13 20:08:41.471172
8196 6 2018-07-13 20:08:41.839193
8197 18 2018-07-13 20:08:42.667246
8198 10 2018-07-13 20:08:43.517295
8199 5 2018-07-13 20:08:43.893318
8200 16 2018-07-13 20:08:45.686421
Estimate Route for the  8200 th household  2018-07-13 20:08:45.778429
8201 11 2018-07-13 20:08:46.105447
8202 6 2018-07-13 20:08:46.466467
8203 10 2018-07-13 20:08:46.866487
8204 6 2018-07-13 20:08:47.227508
8205 8 2018-07-13 20:08:47.660539
8206 17 2018-07-13 20:08:48.270570
8207 4 2018-07-13 20:08:48.634596
8208 11 2018-07-13 20:08:49.252627
8209 15 2018-07-13 20:08:49.873664
8210 7 2018-07-13 20:08:50.224686
8211 5 2018-07-13 20:08:50.523706
8212 7 2018-07-13 20:08:50.865725

In [ ]:
output_flag=1
target_hh_id=2287301
# target_hh_id=2744264
min_length=15
max_length=20
run_mode=0
single_model_runtime=60*5
drivingcost_per_mile=1
reward_mode=0
darp_solutions1=[]
cav_use_mode=1
target_hh=traveler_trips[traveler_trips['hh_id']==target_hh_id].drop_duplicates(subset=['orig_maz','dest_maz','orig_purpose','dest_purpose','starttime','joint_trip_flag'])
#Sort all trips based on start time. This step could reduce the solving time and make it easier to track
sorted_trips=target_hh.sort_values("starttime")
#hh_index give an index to all trips within the household for tracking purpose
hh_num_trips=sorted_trips.shape[0]
sorted_trips["hh_index"]=(range(hh_num_trips))
# sorted_trips=sorted_trips.loc[sorted_trips.tripmode<=6]

# for num_cav in [2]: #range(1,4):
#     for share_ride_factor in [1]: #[1,1.25,1.5]:
#         for time_window_flag in [0]:
num_cav=2
share_ride_factor=1.5
time_window_flag=0
#         time_window_flag=1
# num_cav=1
# share_ride_factor=1.5
# time_window_flag=0
print('**************')
print('Num_CAV',num_cav,share_ride_factor,time_window_flag)
print('**************')
darp_solutions1.extend([sod.solve_with_schedule_partition(sorted_trips,Vehicular_Skim,Transit_AB_Cost_Skim,superzone_map,min_length,max_length,
                            reward_mode,drivingcost_per_mile,share_ride_factor,output_flag,run_mode,num_cav,
                          cav_use_mode,time_window_flag,single_model_runtime)])

In [ ]:
#!/usr/bin/env python
import random
import pylab
from matplotlib.pyplot import pause

pylab.ion()
Gnormal=nx.Graph()
color_map=[]
for i in node_detail:
    color_map.append('blue')
# nx.draw_networkx_nodes(Gnormal, pos,node_list=[1,2,4],node_color='red',node_size=4)
def get_fig():
    global color_map
    for nodeinf in node_detail:
        node=nodeinf[0,0]
        Gnormal.add_node(node,pos=(nodexy[node][0],nodexy[node][1]))
    for linkinf in link_detail: 
        Gnormal.add_edge(linkinf[0,0],linkinf[0,1])

    pos=nx.get_node_attributes(Gnormal,'pos')
    nx.draw(Gnormal,pos,node_size=10,node_color=color_map )
num_plots = 50;
pylab.show()
for i in range(num_plots):
    color_map[i]='red'
    get_fig()
    pylab.draw()
    pause(2)

In [ ]:
target_hh=traveler_trips[traveler_trips['hh_id']==target_hh_id].drop_duplicates(subset=['orig_maz','dest_maz','orig_purpose','dest_purpose','starttime','joint_trip_flag'])
#Sort all trips based on start time. This step could reduce the solving time and make it easier to track
sorted_trips=target_hh.sort_values("starttime")
#hh_index give an index to all trips within the household for tracking purpose
hh_num_trips=sorted_trips.shape[0]

In [ ]:
analyzed_results4=pd.DataFrame()
for solution in darp_solutions4:
        analyzed_results4=analyzed_results4.append(pod.analysis_result(solution,sorted_trips,Vehicular_Skim,superzone_map),ignore_index=True)

In [ ]:
analyzed_results

In [ ]:
for i in range(1,4):
    line_label=str(i)+' CAV'
    plt.pyplot.plot(analyzed_results.loc[analyzed_results.num_cav==i,'share_ride_factor'],analyzed_results.loc[analyzed_results.num_cav==i,'total_AV_driving_time'],'-o',label=line_label)
plt.pyplot.legend()

In [ ]:
sorted_trips.head()

In [ ]:
for i in range(1,4):
    line_label=str(i)+' CAV'
    plt.pyplot.plot(analyzed_results.loc[analyzed_results.num_cav==i,'share_ride_factor'],analyzed_results.loc[analyzed_results.num_cav==i,'total_AV_driving_distance'],'-o',label=line_label)
plt.pyplot.legend()

In [ ]:
analyzed_results.columns

In [ ]:
analyzed_results4

In [ ]:
len(darp_solutions4[0]['route_info'])

In [ ]:
analyzed_results4.loc[analyzed_results4.share_ride_factor==1].transpose()

In [ ]:
zz.hh_vehicle_id.unique()

In [ ]:
for i in range(3):
    print(len(zz.loc[zz.hh_vehicle_id==i]))

In [ ]:
zz=darp_solutions4[-1]['route_info']
# zz=darp_solutions4[5]['route_info']
# zz['origin_network_node','dest_network_node']=zz.apply(lambda row: dy.find_orign_destination_node(row['orig_zone'],row['dest_zone'],origins,destinations))
for car_id in range(3):
    nodes=[]
    for index, row in zz.loc[zz.hh_vehicle_id==car_id].iterrows():
        nodes.extend(dy.find_orign_destination_node(row['orig_zone'],row['dest_zone'],origins,destinations))
    av.zone_node_highlight(nodes,node_detail,link_detail,nodexy,'r')

In [ ]:
darp_solutions4[-1]['route_info'].head()

In [ ]:
pod.plot_route_info_schedule(darp_solutions4[-1]['route_info'],sorted_trips,3)

In [ ]:
analyzed_results4.columns

In [ ]:
# for factor in ['num_cav_trips','num_shared_trips','num_unoccupied_trips','total_schedule_penalty','total_travel_cost','num_pickup_trips']:
factor='total_schedule_penalty'
plt.pyplot.figure()
plt.pyplot.xlabel('Ridesharing Tolerance Factor')
plt.pyplot.ylabel('Total Late/Early Arrival')
for i in range(1,4):
    line_label=str(i)+' AV'
    plt.pyplot.plot(analyzed_results4.loc[analyzed_results4.num_cav==i,'share_ride_factor'],analyzed_results4.loc[analyzed_results4.num_cav==i,'Total_early_time'].apply(lambda x: abs(x))+analyzed_results4.loc[analyzed_results4.num_cav==i,'total_delayed_time'].apply(lambda x: abs(x)),'-o',label=line_label)
plt.pyplot.legend()

In [ ]:
# for factor in ['num_cav_trips','num_shared_trips','num_unoccupied_trips','total_schedule_penalty','total_travel_cost','num_pickup_trips']:
factor='total_travel_cost'
plt.pyplot.figure()
plt.pyplot.xlabel('Ridesharing Tolerance Factor ')
plt.pyplot.ylabel('Total Generalized Travel Cost')
for i in range(1,4):
    line_label=str(i)+' AV'
    plt.pyplot.plot(analyzed_results4.loc[analyzed_results4.num_cav==i,'share_ride_factor'],analyzed_results4.loc[analyzed_results4.num_cav==i,factor].apply(lambda x: abs(x)),'-o',label=line_label)
plt.pyplot.legend()

In [ ]:
len(darp_solutions4[-1]['route_info'])

In [ ]:
for factor in ['num_cav_trips','num_shared_trips','num_unoccupied_trips','total_schedule_penalty','total_travel_cost','num_pickup_trips']:
    plt.pyplot.figure()
    plt.pyplot.xlabel('share ride factor')
    plt.pyplot.ylabel(factor)
    for i in range(1,4):
        line_label=str(i)+' Cav'
        plt.pyplot.plot(analyzed_results.loc[analyzed_results.num_cav==i,'share_ride_factor'],analyzed_results.loc[analyzed_results.num_cav==i,factor].apply(lambda x: abs(x)),'-o',label=line_label)
    plt.pyplot.legend()

In [ ]:
for i in darp_solutions:
    print(i['num_cav'],i['share_ride_factor'])

In [ ]:

route_info.apply(lambda row: prd.estimate_single_car_trip_cost(row.orig_zone,row.dest_zone,row.start_time,row.value_of_time,Vehicular_Skim,2,superzone_map,drivingcost_per_mile),axis=1).sum()/60
route_info.apply(lambda row:prd.estimate_single_car_trip_cost(row.orig_zone,row.dest_zone,row.start_time,row.value_of_time,Vehicular_Skim,1,superzone_map,drivingcost_per_mile) ,axis=1).sum()

In [ ]:
from Modules import Preprocess_DARP as prd

In [ ]:
for j in [2,5,8] :   
    z=darp_solutions4[j]['route_info']
    for i in range(3):
        zz=z.loc[z.hh_vehicle_id==i]
        if len(zz)>0:
            total_driving_time_temp=zz.apply(lambda row: prd.estimate_single_car_trip_cost(row.orig_zone,row.dest_zone,row.start_time,row.value_of_time,Vehicular_Skim,2,superzone_map,drivingcost_per_mile),axis=1).sum()/60
            total_driving_distance_temp=zz.apply(lambda row: prd.estimate_single_car_trip_cost(row.orig_zone,row.dest_zone,row.start_time,row.value_of_time,Vehicular_Skim,1,superzone_map,drivingcost_per_mile),axis=1).sum()
            num_pickup_trips=len(zz.loc[zz.orig_node_index<=len(sorted_trips)])
            print(i,j,len(zz),num_pickup_trips,total_driving_time_temp,total_driving_distance_temp)

In [ ]:
plt.pyplot.figure(1,figsize=[14.5,10])
plt.pyplot.subplot(131)
pod.plot_route_info_schedule(darp_solutions4[2]['route_info'],sorted_trips,darp_solutions4[2]['num_cav'])
plt.pyplot.subplot(132)
pod.plot_route_info_schedule(darp_solutions4[5]['route_info'],sorted_trips,darp_solutions4[5]['num_cav'])
plt.pyplot.subplot(133)
pod.plot_route_info_schedule(darp_solutions4[8]['route_info'],sorted_trips,darp_solutions4[8]['num_cav'])



In [ ]:
plt.pyplot.figure(1,figsize=[14.5,10])
plt.pyplot.subplot(131)
pod.plot_route_info_schedule(darp_solutions[2]['route_info'],sorted_trips,darp_solutions[2]['num_cav'])
plt.pyplot.subplot(132)
pod.plot_route_info_schedule(darp_solutions[5]['route_info'],sorted_trips,darp_solutions[5]['num_cav'])
plt.pyplot.subplot(133)
pod.plot_route_info_schedule(darp_solutions[8]['route_info'],sorted_trips,darp_solutions[8]['num_cav'])


In [ ]:
Vehicular_Skim.loc[18,15,1,1]

In [ ]:
zz1=solution['route_info']

In [ ]:
zz1.dest_zone=zz1.dest_zone.apply(lambda x: int(x))

In [ ]:
analyzed_results3=pd.DataFrame()
for solution in darp_solutions3:
    analyzed_results3=analyzed_results3.append(pod.analysis_result(solution,sorted_trips,Vehicular_Skim,superzone_map),ignore_index=True)

In [ ]:
analyzed_results3.transpose().to_csv('Output/hh_level_result3.csv')

In [ ]:
analyzed_results.transpose().to_csv('hh_level_result.csv')

In [ ]:
analyzed_results.transpose()

### Basic Statistics

In [ ]:
traveler_trips.head()

In [ ]:
traveler_trips.columns

In [ ]:
#Num of household
print('Number of household: \t',len(traveler_trips.hh_id.unique()))
print('Number of traveler: \t',len(traveler_trips.person_id.unique()))
print('Number of trips: \t',len(traveler_trips))
print('Number of indiv trip: \t',len(traveler_trips[traveler_trips.joint_trip_flag==0]))
print('Number of joint trip: \t',len(traveler_trips[traveler_trips.joint_trip_flag==1]))

In [ ]:
traveler_trips['orig_purpose'].value_counts().plot(kind='bar',title='Histogram of Origin Purpose')

In [ ]:
traveler_trips.tripmode.hist()

In [ ]:
traveler_trips.num_trips.hist()
plt.pyplot.title('Histogram of trip per person')

In [ ]:
traveler_trips.groupby('hh_id')['num_trips'].count().hist()
plt.pyplot.title('Histogram of trip per household')

### Sample Household
Select household 2287301 for intial analysis

In [ ]:
target_hh_id=2287301
people_in_hh=traveler_trips[traveler_trips['hh_id']==target_hh_id]['person_id'].unique()
for person in people_in_hh:
    #creat trip-chain-origin-destination-list
    num_trips=traveler_trips[traveler_trips['person_id']==person]['num_trips'].iloc[0]
    print(num_trips)
    trip_chain_or=traveler_trips[traveler_trips['person_id']==person][['origin_node','destination_node']]
    trip_chain_or['origin_node'].iloc[1:num_trips-1]=trip_chain_or['destination_node'].iloc[0:num_trips-2].tolist()
    trip_chain_or['origin_node'].iloc[num_trips-1]=trip_chain_or['destination_node'].iloc[num_trips-2]
    #draw the graph
    av.trip_chain_highlight(trip_chain_or,node_detail,link_detail,nodexy)
#traveler_trips[traveler_trips['hh_id']==2105189]

### Solve the optimization for a household 

The factors need to know about:  
1. Decision variable X <sub>ij</sub> and T<sub>j</sub> (departure time from node j)  
2. R <sub>i</sub> reward of visiting node i
3. S <sub>i</sub> penalty for late/early arrival
4. C <sub>ij</sub> travel time between node i and j  
5. 

#### Solve with heuristic

## Estimate travel time and cost

## Solve the optimization with Gurobi

In [ ]:
drivingcost_per_mile=1
transit_fare=2
transit_asc=2

##### Try to solve the problem for a single household

#### Solve with exact method

In [ ]:
print('prepare input for dial n ride model at ',datetime.datetime.now())
output_flag=1
target_hh_id=2287301
target_hh=traveler_trips[traveler_trips['hh_id']==target_hh_id].drop_duplicates(subset=['orig_maz','dest_maz','orig_purpose','dest_purpose','starttime','joint_trip_flag'])
#Sort all trips based on start time. This step could reduce the solving time and make it easier to track
sorted_trips=target_hh.sort_values("starttime")
#hh_index give an index to all trips within the household for tracking purpose
hh_num_trips=sorted_trips.shape[0]
sorted_trips["hh_index"]=(range(hh_num_trips))
num_hh_member,hh_num_trips,C,TT,expected_arrival_time,expected_leave_time,early_penalty,late_penalty,early_penalty_threshold,late_penalty_threshold,visit_candidate_zone=extract_hh_information(sorted_trips,Vehicular_Skim,Transit_AB_Cost_Skim,superzone_map)
# R=estimate_transit_cost(sorted_trips,TransitMazTazFlag,WalkSpeed,TransitSkimTimeIntervalLength,Transit_AB_Cost_Skim,Transit_AB_Time_Skim,transit_zone_candidates,three_link_walk)
R=estimate_trip_reward(hh_num_trips,sorted_trips,Vehicular_Skim)
print('start sovling problem at ',datetime.datetime.now())
m1,x,T=dial_n_ride_model(num_hh_member,hh_num_trips,C,TT,sorted_trips,expected_arrival_time,early_penalty,late_penalty,early_penalty_threshold,late_penalty_threshold,R,share_ride_factor,output_flag,run_mode=)
print('finish solving problem at ',datetime.datetime.now())
route_info=extract_route_from_model_solution(x,T,sorted_trips,visit_candidate_zone,hh_num_trips,expected_arrival_time,expected_leave_time)
print('extract solution at',datetime.datetime.now())
route_info.to_csv('route_info_yingjie.csv')

In [ ]:
T_sol=np.ones(2*hh_num_trips+2)
for i in range(2*hh_num_trips+2):
#     print(int(T[i].x),'\t',expected_arrival_time[i],'\t',T[i].x-expected_arrival_time[i])
    T_sol[i]=T[i].x
schedule_deviation=T_sol-expected_arrival_time
analysis_result(all_trips_route_info_partition,sorted_trips,Vehicular_Skim,schedule_deviation)

#### Solve with the schedule partition heuristic

In [ ]:
def analysis_result(route_info,sorted_trips,Vehicular_Skim,schedule_deviation):
    print('Number of CAV Trips',len(route_info))
    print('Number of Occupied trips',len(route_info.loc[route_info.person_id>0]))
    print('Number of unoccupied trips',len(route_info.loc[route_info.person_id==0]))
    print('Number of pickup trips',len(route_info.loc[route_info.orig_node_index<=len(sorted_trips)]))
    print('Number of shared trips',len(route_info.loc[(route_info.orig_node_index<=len(sorted_trips)) &(route_info.dest_node_index<=len(sorted_trips))]))
    print('Number of convention car trips',len(sorted_trips.loc[sorted_trips.tripmode<=6]))
    # Compare the VHT between convention vehicle and CAV
    
    total_convention_vehicle_driving_time=sorted_trips.loc[sorted_trips.tripmode<=6].apply(lambda row:estimate_single_car_trip_cost(row.orig_taz,row.dest_taz,row.starttime,row.value_of_time,Vehicular_Skim,2),axis=1).sum()
    total_AV_driving_time=route_info.apply(lambda row: estimate_single_car_trip_cost(row.orig_zone,row.dest_zone,row.start_time,row.value_of_time,Vehicular_Skim,2),axis=1).sum()
    total_AV_unoccupied_driving_time=route_info.loc[route_info.person_id==0].apply(lambda row: estimate_single_car_trip_cost(row.orig_zone,row.dest_zone,row.start_time,row.value_of_time,Vehicular_Skim,2),axis=1).sum()
    print('Convention car trips VHT: \t',total_convention_vehicle_driving_time/60,
          '\nAV driving time: \t',total_AV_driving_time/60,
          '\nAV unoccupied driving_time: \t',total_AV_unoccupied_driving_time/60)
    # Compare the VMT between convention vehicle and CAV
    total_convention_vehicle_driving_distance=sorted_trips.loc[sorted_trips.tripmode<=6].apply(lambda row: estimate_single_car_trip_cost(row.orig_taz,row.dest_taz,row.starttime,row.value_of_time,Vehicular_Skim,1),axis=1).sum()
    total_AV_driving_distance=route_info.apply(lambda row:estimate_single_car_trip_cost(row.orig_zone,row.dest_zone,row.start_time,row.value_of_time,Vehicular_Skim,1) ,axis=1).sum()
    total_AV_unoccupied_driving_distance=route_info.loc[route_info.person_id==0].apply(lambda row:estimate_single_car_trip_cost(row.orig_zone,row.dest_zone,row.start_time,row.value_of_time,Vehicular_Skim,1),axis=1).sum()
    print('Convention VMT: \t',total_convention_vehicle_driving_distance,
          '\nAV driving distance: \t',total_AV_driving_distance,
          '\nAV unoccupied driving_distance: \t',total_AV_unoccupied_driving_distance)
    print('Number of delayed trips\t',sum(1 for i in schedule_deviation if i >1))
    print('Number of early trips\t',sum(1 for i in schedule_deviation if i <-1))
    print('Total delayed time\t',sum(i for i in schedule_deviation if i >0))
    print('Total_early_time\t',sum(i for i in schedule_deviation if i <0))
    plt.pyplot.figure(1,figsize=[2.3,10])
    plot_route_info_schedule(route_info,sorted_trips)
    return

In [ ]:
def plot_route_info_schedule(route_info,sorted_trips):
#     z=traveler_trips[traveler_trips.hh_id==route_info.hh_id[0]]
    sorted_trips.loc[:,'p_id']=sorted_trips.groupby(['person_id']).ngroup()
    hh_num_trips=len(sorted_trips)
    person_id_and_inhouse_p_id_map=dict(zip(sorted_trips.person_id,sorted_trips.p_id))
    person_id_and_inhouse_p_id_map[0]=-1
    route_info['p_id']=route_info.person_id.apply(lambda x: person_id_and_inhouse_p_id_map[x])
    
    plt.pyplot.scatter(route_info.loc[(route_info.p_id!=-1) & (route_info.orig_node_index<1+hh_num_trips),'p_id'],
                route_info.loc[(route_info.p_id!=-1) & (route_info.orig_node_index<1+hh_num_trips),'origin_arrival_time'])
    plt.pyplot.plot(route_info.loc[(route_info.p_id!=-1) & (route_info.orig_node_index<1+hh_num_trips),'p_id'],
                    route_info.loc[(route_info.p_id!=-1) & (route_info.orig_node_index<1+hh_num_trips),'origin_arrival_time'])
#     plt.pyplot.scatter(route_info.loc[(route_info.p_id!=-1) & (route_info.orig_node_index<1+hh_num_trips) & (route_info.dest_node_index<1+hh_num_trips),'p_id'],
#                     route_info.loc[(route_info.p_id!=-1) & (route_info.orig_node_index<1+hh_num_trips) &(route_info.dest_node_index<1+hh_num_trips),'origin_arrival_time'])
    plt.pyplot.xlabel('Traveler Index')
    plt.pyplot.ylabel('Time')
    plt.pyplot.grid()
    plt.pyplot.xticks(np.arange(min(sorted_trips.p_id), max(sorted_trips.p_id)+1, 1))
    plt.pyplot.yticks(np.arange(0,1441,30))
#     plt.pyplot.title(title)
    return 

#### All trips take CAV

In [ ]:
output_flag=1
target_hh_id=2287301
# target_hh_id=2744264
min_length=15
max_length=20
run_mode=0
reward_mode=0
share_ride_factor=2
time_window_flag=0
single_model_runtime=60*3
target_hh=traveler_trips[traveler_trips['hh_id']==target_hh_id].drop_duplicates(subset=['orig_maz','dest_maz','orig_purpose','dest_purpose','starttime','joint_trip_flag'])
#Sort all trips based on start time. This step could reduce the solving time and make it easier to track
sorted_trips=target_hh.sort_values("starttime")
#hh_index give an index to all trips within the household for tracking purpose
hh_num_trips=sorted_trips.shape[0]
sorted_trips["hh_index"]=(range(hh_num_trips))
all_trips_route_info_partition,all_trips_schedule_deviation=sod.solve_with_schedule_partition(sorted_trips,Vehicular_Skim,Transit_AB_Cost_Skim,superzone_map,min_length,max_length)

In [ ]:
analysis_result(all_trips_route_info_partition,sorted_trips,Vehicular_Skim,all_trips_schedule_deviation)

#### Strict time window

In [ ]:
output_flag=1
target_hh_id=2287301
# target_hh_id=2744264
min_length=15
max_length=20
run_mode=0
reward_mode=2
time_window_flag=1
num_poav=2
single_model_runtime=60*5
target_hh=traveler_trips[traveler_trips['hh_id']==target_hh_id].drop_duplicates(subset=['orig_maz','dest_maz','orig_purpose','dest_purpose','starttime','joint_trip_flag'])
#Sort all trips based on start time. This step could reduce the solving time and make it easier to track
sorted_trips=target_hh.sort_values("starttime")
#hh_index give an index to all trips within the household for tracking purpose
hh_num_trips=sorted_trips.shape[0]
sorted_trips["hh_index"]=(range(hh_num_trips))
all_trips_route_info_stricttw,all_trips_schedule_deviation_stricttw=solve_with_schedule_partition(sorted_trips,Vehicular_Skim,Transit_AB_Cost_Skim,superzone_map,min_length,max_length)

In [ ]:
analysis_result(all_trips_route_info_stricttw,sorted_trips,Vehicular_Skim,all_trips_schedule_deviation_stricttw)

##### Only car trips take CAV

In [ ]:
len(sorted_car_trips)

In [ ]:
output_flag=1
target_hh_id=2287301
# target_hh_id=2744264
min_length=15
max_length=20
run_mode=0
reward_mode=0
time_window_flag=0
single_model_runtime=60*3
target_hh=traveler_trips[traveler_trips['hh_id']==target_hh_id].drop_duplicates(subset=['orig_maz','dest_maz','orig_purpose','dest_purpose','starttime','joint_trip_flag'])
#Sort all trips based on start time. This step could reduce the solving time and make it easier to track
sorted_trips=target_hh.sort_values("starttime")
sorted_car_trips=sorted_trips.loc[sorted_trips.tripmode<=6]
#hh_index give an index to all trips within the household for tracking purpose
hh_num_trips=sorted_car_trips.shape[0]
sorted_car_trips["hh_index"]=(range(hh_num_trips))
car_trips_route_info_partition,car_trips_schedule_deviation=solve_with_schedule_partition(sorted_car_trips,Vehicular_Skim,Transit_AB_Cost_Skim,superzone_map,min_length,max_length)

In [ ]:
analysis_result(car_trips_route_info_partition,sorted_car_trips,Vehicular_Skim,car_trips_schedule_deviation)

#### Ban Shared Rides

In [ ]:
output_flag=1
target_hh_id=2287301
# target_hh_id=2744264
min_length=15
max_length=20
target_hh=traveler_trips[traveler_trips['hh_id']==target_hh_id].drop_duplicates(subset=['orig_maz','dest_maz','orig_purpose','dest_purpose','starttime','joint_trip_flag'])
#Sort all trips based on start time. This step could reduce the solving time and make it easier to track
sorted_trips=target_hh.sort_values("starttime")
# sorted_trips=sorted_trips.loc[sorted_trips.tripmode<=6]
#hh_index give an index to all trips within the household for tracking purpose
hh_num_trips=sorted_trips.shape[0]
sorted_trips["hh_index"]=(range(hh_num_trips))
all_trips_route_info_ban_sharerides,all_trips_schedule_deviation_ban_sharerides=solve_with_schedule_partition(sorted_trips,Vehicular_Skim,Transit_AB_Cost_Skim,superzone_map,min_length,max_length)

In [ ]:
analysis_result(all_trips_route_info_ban_sharerides,sorted_trips,Vehicular_Skim,all_trips_schedule_deviation_ban_sharerides)

#### Share ride factor 1.25

In [ ]:
output_flag=1
target_hh_id=2287301
# target_hh_id=2744264
min_length=15
max_length=20
share_ride_factor=1.25
target_hh=traveler_trips[traveler_trips['hh_id']==target_hh_id].drop_duplicates(subset=['orig_maz','dest_maz','orig_purpose','dest_purpose','starttime','joint_trip_flag'])
#Sort all trips based on start time. This step could reduce the solving time and make it easier to track
sorted_trips=target_hh.sort_values("starttime")
# sorted_trips=sorted_trips.loc[sorted_trips.tripmode<=6]
#hh_index give an index to all trips within the household for tracking purpose
hh_num_trips=sorted_trips.shape[0]
sorted_trips["hh_index"]=(range(hh_num_trips))
all_trips_route_info_sharerides125,all_trips_schedule_deviation_sharerides125=solve_with_schedule_partition(sorted_trips,Vehicular_Skim,Transit_AB_Cost_Skim,superzone_map,min_length,max_length)

In [ ]:
analysis_result(all_trips_route_info_sharerides125,sorted_trips,Vehicular_Skim,all_trips_schedule_deviation_sharerides125)

In [ ]:
plt.pyplot.figure(1,figsize=[21,10])
plt.pyplot.subplot(141)
plt.pyplot.scatter(sorted_trips.person_id.apply(lambda x: person_id_and_inhouse_p_id_map[x]),sorted_trips.starttime)
plt.pyplot.scatter(sorted_trips.loc[(sorted_trips.tripmode<=6),'p_id'],
                   sorted_trips.loc[(sorted_trips.tripmode<=6),'starttime'],
                   c='r')
plt.pyplot.xlabel('Traveler Index')
plt.pyplot.ylabel('Time')
plt.pyplot.grid()
plt.pyplot.xticks(np.arange(min(sorted_trips.p_id), max(sorted_trips.p_id)+1, 1))
plt.pyplot.yticks(np.arange(0,1441,30))
plt.pyplot.subplot(142)
plot_route_info_schedule(car_trips_route_info_partition,sorted_car_trips)
plt.pyplot.subplot(143)
plot_route_info_schedule(all_trips_route_info_partition,sorted_trips)
plt.pyplot.subplot(144)
plot_route_info_schedule(all_trips_route_info_ban_sharerides,sorted_trips)

#### Probability Mode Choice

In [ ]:
drivingcost_per_mile=1
transit_fare=1.4
transit_asc=2

In [ ]:
output_flag=1
target_hh_id=2287301
# target_hh_id=2744264
min_length=15
max_length=20
run_mode=1
share_ride_factor=1.5
target_hh=traveler_trips[traveler_trips['hh_id']==target_hh_id].drop_duplicates(subset=['orig_maz','dest_maz','orig_purpose','dest_purpose','starttime','joint_trip_flag'])
#Sort all trips based on start time. This step could reduce the solving time and make it easier to track
sorted_trips=target_hh.sort_values("starttime")
# sorted_trips=sorted_trips.loc[sorted_trips.tripmode<=6]
#hh_index give an index to all trips within the household for tracking purpose
hh_num_trips=sorted_trips.shape[0]
sorted_trips["hh_index"]=(range(hh_num_trips))
mode_choice_route_info,mode_choice_schedule_deviation=solve_with_schedule_partition(sorted_trips,Vehicular_Skim,Transit_AB_Cost_Skim,superzone_map,min_length,max_length)

In [ ]:
sorted_trips=compare_mode_utlity(sorted_trips)

In [ ]:
min_length=15
max_length=20
run_mode=0
share_ride_factor=1.5
force_serve_factor=1
car_sorted_trips=sorted_trips.loc[sorted_trips.expected_mode=='Car']
# sorted_trips=sorted_trips.loc[sorted_trips.tripmode<=6]
#hh_index give an index to all trips within the household for tracking purpose
hh_num_trips=sorted_trips.shape[0]
sorted_trips["hh_index"]=(range(hh_num_trips))
car_only_mode_choice_route_info,car_only_mode_choice_schedule_deviation=solve_with_schedule_partition(car_sorted_trips,Vehicular_Skim,Transit_AB_Cost_Skim,superzone_map,min_length,max_length)
force_serve_factor=0

In [ ]:
sorted_trips[['transit_time','transit_walk_time','car_time','toll_cost','car_dist','transit_utility','car_utility','expected_mode','actual_mode','value_of_time']]

In [ ]:
# sub_sorted_trips=[item for sublist in schedule_partition(sorted_trips,Vehicular_Skim,min_length,max_length) for item in sublist]
# sub_sorted_trips=flatten(schedule_partition(sorted_trips,Vehicular_Skim,min_length,max_length))
# route_info=pd.DataFrame()

# total_previous_sub_trips_length=0
# total_tailing_sub_trips_length=0
# schedule_deviation=[]
# x_list=[]
# for sub_sorted_trip in sub_sorted_trips:

#     num_hh_member,hh_num_trips,C,TT,expected_arrival_time,expected_leave_time,early_penalty,late_penalty,early_penalty_threshold,late_penalty_threshold,visit_candidate_zone=extract_hh_information(sub_sorted_trip,Vehicular_Skim,Transit_AB_Cost_Skim,superzone_map)
#     # R=estimate_transit_cost(sorted_trips,TransitMazTazFlag,WalkSpeed,TransitSkimTimeIntervalLength,Transit_AB_Cost_Skim,Transit_AB_Time_Skim,transit_zone_candidates,three_link_walk)
#     R=estimate_trip_reward(hh_num_trips,sub_sorted_trip,Vehicular_Skim)
#     print('start sovling problem at ',datetime.datetime.now())
#     m1,x,T=dial_n_ride_model(num_hh_member,hh_num_trips,C,TT,sub_sorted_trip,expected_arrival_time,early_penalty,late_penalty,early_penalty_threshold,late_penalty_threshold,R,output_flag)
#     x_list.extend([x])
#     print('finish solving problem at ',datetime.datetime.now())
#     sub_route_info=extract_route_from_model_solution(x,T,sub_sorted_trip,visit_candidate_zone,hh_num_trips,expected_arrival_time,expected_leave_time)
#     total_tailing_sub_trips_length=len(sorted_trips)-total_previous_sub_trips_length-len(sub_sorted_trip)
#     sub_route_info['orig_node_index']=sub_route_info.orig_node_index.apply(lambda x: x+total_previous_sub_trips_length if x<=hh_num_trips else x+2*total_previous_sub_trips_length+total_tailing_sub_trips_length) 
#     sub_route_info['dest_node_index']=sub_route_info.dest_node_index.apply(lambda x: x+total_previous_sub_trips_length if x<=hh_num_trips else x+2*total_previous_sub_trips_length+total_tailing_sub_trips_length) 
#     total_previous_sub_trips_length=total_previous_sub_trips_length+len(sub_sorted_trip)
#     route_info=route_info.append(sub_route_info)
# #         for index, row in route_info.iterrows():
# #             print(route_info.dest_expected_arrival_time,'\t',row.dest_arrival_time,'\t',row.start_time,'\t',T[row.dest_node_index].x)
# #         #Estimate the delay and early arrival
# T_sol=np.ones(2*hh_num_trips+2)
# for i in range(2*hh_num_trips+2):
# #     print(int(T[i].x),'\t',expected_arrival_time[i],'\t',T[i].x-expected_arrival_time[i])
#     T_sol[i]=T[i].x
#     print(T_sol[i],expected_arrival_time[i])
# schedule_deviation.extend(T_sol-expected_arrival_time)

In [ ]:
sorted_trips.loc[:,'p_id']=sorted_trips.groupby(['person_id']).ngroup()
hh_num_trips=len(sorted_trips)
person_id_and_inhouse_p_id_map=dict(zip(sorted_trips.person_id,sorted_trips.p_id))
person_id_and_inhouse_p_id_map[0]=-1
plt.pyplot.figure(1,figsize=[21,10])
plt.pyplot.subplot(141)
plot_route_info_schedule(car_only_mode_choice_route_info,car_sorted_trips)
z1=[sorted_trips.iloc[i-1].p_id for i in temp3]
z2=[sorted_trips.iloc[i-1].starttime for i in temp3]
print(z1,z2)
plt.pyplot.scatter(z1,z2)

In [ ]:
analysis_result(mode_choice_route_info,sorted_trips,Vehicular_Skim,mode_choice_schedule_deviation)

In [ ]:
sub_sorted_trips=flatten(schedule_partition(sorted_trips,Vehicular_Skim,min_length,max_length))
#     route_info=pd.DataFrame()
for sub_sorted_trip in sub_sorted_trips:
#     sub_sorted_trip=sub_sorted_trips[1]
    num_hh_member,hh_num_trips,C,TT,expected_arrival_time,expected_leave_time,early_penalty,late_penalty,early_penalty_threshold,late_penalty_threshold,visit_candidate_zone=extract_hh_information(sub_sorted_trip,Vehicular_Skim,Transit_AB_Cost_Skim,superzone_map)
    # R=estimate_transit_cost(sorted_trips,TransitMazTazFlag,WalkSpeed,TransitSkimTimeIntervalLength,Transit_AB_Cost_Skim,Transit_AB_Time_Skim,transit_zone_candidates,three_link_walk)
    # R=np.ones(hh_num_trips+1)*10
    R=estimate_trip_reward(hh_num_trips,sub_sorted_trip,Vehicular_Skim)
    print('start sovling problem at ',datetime.datetime.now())
    m1,x,T=dial_n_ride_model(num_hh_member,hh_num_trips,C,TT,sub_sorted_trip,expected_arrival_time,early_penalty,late_penalty,early_penalty_threshold,late_penalty_threshold,R,share_ride_factor,output_flag,run_mode)
    print('finish solving problem at ',datetime.datetime.now())
    sub_route_info_temp=extract_route_from_model_solution(x,T,sub_sorted_trip,visit_candidate_zone,hh_num_trips,expected_arrival_time,expected_leave_time)
    
    T_sol=np.ones(2*hh_num_trips+2)
    for i in range(2*hh_num_trips+2):
    #     print(int(T[i].x),'\t',expected_arrival_time[i],'\t',T[i].x-expected_arrival_time[i])
        T_sol[i]=T[i].x
    schedule_deviation.extend(T_sol-expected_arrival_time)
    analysis_result(sub_route_info_temp,sub_sorted_trips,Vehicular_Skim,schedule_deviation)

In [ ]:
analysis_result(sub_route_info,sub_sorted_trip,Vehicular_Skim,schedule_deviation)

In [ ]:
z=traveler_trips[traveler_trips.hh_id==2287301]
z.loc[:,'p_id']=z.groupby(['person_id']).ngroup()
z.loc[:,'expected_arrival']=z.starttime+z.travel_time
person_id_and_inhouse_p_id_map=dict(zip(z.person_id,z.p_id))
person_id_and_inhouse_p_id_map[0]=-1
sorted_trips.loc[:,'p_id']=sorted_trips.person_id.apply(lambda x: person_id_and_inhouse_p_id_map[x])
route_info['p_id']=route_info.person_id.apply(lambda x: person_id_and_inhouse_p_id_map[x])
route_info_partition['p_id']=route_info_partition.person_id.apply(lambda x: person_id_and_inhouse_p_id_map[x])

plt.pyplot.figure(1,figsize=[21,14])
plt.pyplot.subplot(131)
plt.pyplot.scatter(sorted_trips.person_id.apply(lambda x: person_id_and_inhouse_p_id_map[x]),sorted_trips.starttime)

# plt.pyplot.scatter(z.person_id.apply(lambda x: person_id_and_inhouse_p_id_map[x]),z.starttime,marker='o',facecolors='none', edgecolors='r')

# plt.pyplot.plot(route_info.loc[(route_info.p_id!=-1) & (route_info.orig_node_index<1+hh_num_trips),'p_id'],
#                 route_info.loc[(route_info.p_id!=-1) & (route_info.orig_node_index<1+hh_num_trips),'start_time']
#                )
plt.pyplot.scatter(sorted_trips.loc[(sorted_trips.tripmode<=6),'p_id'],
                   sorted_trips.loc[(sorted_trips.tripmode<=6),'starttime'],
                   c='r')
plt.pyplot.xlabel('Traveler Index')
plt.pyplot.ylabel('Time')
plt.pyplot.grid()
plt.pyplot.xticks(np.arange(min(sorted_trips.p_id), max(sorted_trips.p_id)+1, 1))
plt.pyplot.yticks(np.arange(0,1441,30))
plt.pyplot.subplot(132)
# plt.pyplot.scatter(sorted_trips.person_id.apply(lambda x: person_id_and_inhouse_p_id_map[x]),sorted_trips.starttime)
# plt.pyplot.scatter(sorted_trips.loc[(sorted_trips.tripmode<=6),'p_id'],
#                    sorted_trips.loc[(sorted_trips.tripmode<=6),'starttime'],
#                    c='r')

plt.pyplot.scatter(route_info.loc[(route_info.p_id!=-1) & (route_info.orig_node_index<=1+hh_num_trips),'p_id'],
                route_info.loc[(route_info.p_id!=-1) & (route_info.orig_node_index<=1+hh_num_trips),'origin_arrival_time'])
plt.pyplot.plot(route_info.loc[(route_info.p_id!=-1) & (route_info.orig_node_index<=1+hh_num_trips),'p_id'],
                route_info.loc[(route_info.p_id!=-1) & (route_info.orig_node_index<=1+hh_num_trips),'origin_arrival_time'])
plt.pyplot.grid()
plt.pyplot.xticks(np.arange(min(sorted_trips.p_id), max(sorted_trips.p_id)+1, 1))
plt.pyplot.yticks(np.arange(0,1441,30))
plt.pyplot.subplot(133)
plt.pyplot.scatter(route_info_partition.loc[(route_info_partition.p_id!=-1) & (route_info_partition.orig_node_index<=1+hh_num_trips),'p_id'],
                route_info_partition.loc[(route_info_partition.p_id!=-1) & (route_info_partition.orig_node_index<=1+hh_num_trips),'origin_arrival_time'])
plt.pyplot.plot(route_info_partition.loc[(route_info_partition.p_id!=-1) & (route_info_partition.orig_node_index<=1+hh_num_trips),'p_id'],
                route_info_partition.loc[(route_info_partition.p_id!=-1) & (route_info_partition.orig_node_index<=1+hh_num_trips),'origin_arrival_time'])

plt.pyplot.xlabel('Traveler Index')
plt.pyplot.ylabel('Time')
plt.pyplot.grid()
plt.pyplot.xticks(np.arange(min(sorted_trips.p_id), max(sorted_trips.p_id)+1, 1))
plt.pyplot.yticks(np.arange(0,1441,30))

In [ ]:
sorted_trips=target_hh.sort_values("starttime")
sub_sorted_trips=[item for sublist in schedule_partition(sorted_trips,Vehicular_Skim,min_length,max_length) for item in sublist]

In [ ]:
plot_route_info_schedule(sub_route_info,traveler_trips)

In [ ]:
sub_route_info.groupby('person_id').count()

In [ ]:
sub_sorted_trip.groupby('person_id').count()

In [ ]:
sub_route_info

##### Solve the problem for all households

In [ ]:
route_infos=get_route_info_allhh(traveler_trips,0)

In [ ]:
route_infos.to_csv('Output/route_infos_fullconversion_lessthan40triphh.csv')

In [ ]:
route_infos.to_csv('Output/route_infos_partconversion.csv')

In [ ]:
route_infos_full_adoption=pd.read_csv('D:/alex/Autonomous-V-demand/Output/route_infos_fullconversion.csv')
route_infos_part_adoption=pd.read_csv('D:/alex/Autonomous-V-demand/Output/route_infos_partconversion.csv')

In [ ]:
route_infos_full_adoption.head()

#### Compare the new route with the old one

In [ ]:
initial_trips=traveler_trips.drop_duplicates(subset=['hh_id','orig_maz','dest_maz','orig_purpose','dest_purpose','starttime'])
len1=[]
len2=[]
origin_num_car_trips=[]
num_empty_car_trips=[]
num_initial_car_trips=0
for name, group in route_infos_full_adoption.groupby(['hh_id']):
    num_initial_car_trips=num_initial_car_trips+len(initial_trips.loc[(initial_trips.hh_id==name) & ((initial_trips.tripmode<=6) | (initial_trips.tripmode==13))])
#     len1.extend([len(group)])
#     len2.extend([len(initial_trips.loc[initial_trips.hh_id==name])])
#     origin_num_car_trips.extend([len(initial_trips.loc[(initial_trips.tripmode<=6) | (initial_trips.tripmode==13)])])
#     num_empty_car_trips.extend([len(group[group.person_id==0])])

In [ ]:
zz=traveler_trips.drop_duplicates(subset=['hh_id','orig_maz','dest_maz','orig_purpose','dest_purpose','starttime']).groupby('hh_id').count() 

In [ ]:
zz1=traveler_trips.groupby('hh_id').count() 

In [ ]:
traveler_trips.columns

In [ ]:
zz[zz.person_id>30].person_id.sum()

In [ ]:
print('The new route plan has ',sum(len1),' trips')
print('The origin trip plan has ',sum(len2),' trips')
print('The origin trip plan has',len(initial_trips.loc[initial_trips.tripmode<=6]),'car trips')
print('The origin trip plan has',len(initial_trips.loc[initial_trips.tripmode==13]),'taxi trips')
print('The new route plan has',sum(num_empty_car_trips),'empty car trips')

### Route result analysis

##### Estimate how many trips are satisfied

In [ ]:
route_infos.head()

In [ ]:
def calculate_num_pick_up_trips(route_infos,traveler_trips):
    total_num_pickup_trip=0
    total_num_joint_trip=0
    for target_hh_id in route_infos.hh_id.unique():
        target_hh=traveler_trips[traveler_trips['hh_id']==target_hh_id].drop_duplicates(subset=['orig_maz','dest_maz','orig_purpose','dest_purpose','starttime'])
        #Sort all trips based on start time. This step could reduce the solving time and make it easier to track
        sorted_trips=target_hh.sort_values("starttime")
        hh_num_trips=len(sorted_trips)
        total_num_pickup_trip=total_num_pickup_trip+len(route_infos.loc[(route_infos.hh_id==target_hh_id)& (route_infos.orig_node_index<hh_num_trips)])
        total_num_joint_trip=total_num_joint_trip+len(route_infos.loc[(route_infos.hh_id==target_hh_id)& (route_infos.orig_node_index<hh_num_trips)& (route_infos.dest_node_index<hh_num_trips)])
    return total_num_pickup_trip,total_num_joint_trip
    #     print(target_hh_id)

In [ ]:
# Calculate the number of trips with/without passenger
full_adoption_num_pickup,full_adoption_extra_joint=calculate_num_pick_up_trips(route_infos_full_adoption,traveler_trips)
full_adoption_empty_trips=len(route_infos_full_adoption.loc[route_infos_full_adoption.person_id==0])
partial_adoption_num_pickup,partial_adoption_extra_joint=calculate_num_pick_up_trips(route_infos_part_adoption,traveler_trips)
partial_adoption_empty_trips=len(route_infos_part_adoption.loc[route_infos_part_adoption.person_id==0])

In [ ]:
full_adoption_num_pickup

In [ ]:
partial_adoption_num_pickup

In [ ]:
full_adoption_extra_joint

In [ ]:
len(route_infos_part_adoption)

In [ ]:
partial_adoption_extra_joint

In [ ]:
print('The full conversion plan pickup ',full_adoption_num_pickup,'trips (',round(full_adoption_num_pickup/sum(len2),2),')out of ',sum(len2))
print('The full conversion plan incurs ', full_adoption_empty_trips,'empty trips. The ratio beween empty and occupy trips is', full_adoption_empty_trips/full_adoption_num_pickup)
print('The partial conversion plan pickup ',partial_adoption_num_pickup,'trips (',round(partial_adoption_num_pickup/sum(len2),2),')out of ',sum(len2))
print('The partial conversion plan incurs ', partial_adoption_empty_trips,'empty trips. The ratio beween empty and occupy trips is' ,partial_adoption_empty_trips/partial_adoption_num_pickup)

In [ ]:
route_infos_full_adoption.columns

In [ ]:
z=route_infos_full_adoption.groupby('hh_id').apply(lambda x: (x==0).sum())

In [ ]:
z['num_hh_member']=traveler_trips.groupby('hh_id')['person_id'].nunique()

In [ ]:
def plot_num_trips_by_timeinterval(route_infos):
    route_infos['occupy_flag']=['Occupied' if x >0 else 'Empty' for x in route_infos['person_id']]
    route_infos['start_time_interval']=[math.floor(x/36) for x in route_infos['start_time']]
    z=route_infos.groupby(['start_time_interval', 'occupy_flag'])['start_time'].count().unstack('occupy_flag').fillna(0)
#     fig, axes = plt.pyplot.subplots(nrows=1 ,ncols=1,figsize=(16,6))
    
    z[['Occupied','Empty']].plot(kind='bar',stacked=True,figsize=(16,6))
    z['Empty_percentage']=z['Empty']/(z['Occupied']+z['Empty'])
#     z['Empty_percentage'].plot(kind='bar',ax=axes[1])
    return z

In [ ]:
z1=plot_num_trips_by_timeinterval(route_infos_full_adoption)

In [ ]:
z2=plot_num_trips_by_timeinterval(route_infos_part_adoption)

In [ ]:
fig = plt.pyplot.figure()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
z1.Empty_percentage.plot(kind='bar',ax=ax,color='r',position=0)
z2.Empty_percentage.plot(kind='bar',ax=ax2,color='blue',position=1)

#### Write the route to DYNASMART input file vehicle.dat

#### Result Analysis